<a href="https://colab.research.google.com/github/Satyake/Open-AI-LLM-Langchain-Projects/blob/main/VectorDB_PineCone_Similaritysearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip -q install pypdf langchain sentence-transformers pinecone-client==2.2.4 openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [5]:
!mkdir pdfs

In [6]:
from langchain.document_loaders import PyPDFDirectoryLoader
#extract text from pdf
loader=PyPDFDirectoryLoader('pdfs')
data=loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=50)
text_chunks=text_splitter.split_documents(data)

In [12]:
text_chunks[1]

Document(page_content='BAKSHI  \n \n satyakeb@gmail.com  \n 613-762-4038', metadata={'source': 'pdfs/Bakshi_CV.pdf', 'page': 0})

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
query_result=embeddings.embed_query('hello world')

In [17]:
len(query_result)

384

In [45]:
import os
from google.colab import userdata
pinecone_api=userdata.get('pinecone')
openai=userdata.get('openai')
os.environ['PINECONE_API_KEY']=pinecone_api
os.environ['PINECONE_API_ENV']='gcp-starter'
os.environ['OPENAI_API_KEY']=openai


In [33]:
import pinecone
pinecone.init()

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [34]:
from pinecone import Pinecone

pc=Pinecone(
     api_key='PINECONE_API_KEY',
     environment='PINECONE_API_ENV'
 )
index_name='pinecone-test'

In [36]:
from langchain.vectorstores import Pinecone
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [37]:
#if index is already there
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [38]:
docsearch

In [39]:
#similarity search


In [ ]:
query="what is yolo?"

In [40]:
docs=docsearch.similarity_search(query,k=3)

In [52]:
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
llm=OpenAI()
qa=RetrievalQA.from_chain_type(llm=llm,
                               chain_type='stuff',
                               retriever=docsearch.as_retriever())

In [54]:
query="Can you tell me if this person has Azure certification?"
qa.run(query)

' Yes, it appears that this person has Azure certification. They are listed as having both "Platform (Certified)" and "Azure Machine Learning" in their context.'